# FOOOF: Results Analysis

<div class="alert alert-success">
This tutorial explores some basic examples analyzing the results from fitting FOOOF models.
</div>

<div class="alert alert-info">
To go back to code-based examples of using FOOOF, go [here](02-FOOOF.ipynb).
</div>

## Exploring FOOOF Analyses

FOOOF is really a way to extract features of interest from your data. 

These features can then be examined across or between groups of interest, or perhaps fed into further analysis. 

Largely, it is up to you what to do after running FOOOF, and depends on your questions of interest.

Here, we briefly introduce some analysis utilities that are packaged with FOOOF, and explore some simple analyses that can be done with FOOOF outputs. 

In [1]:
# General imports
import numpy as np

# Import FOOOF objects & synth utilities
from fooof import FOOOF, FOOOFGroup
from fooof.synth import gen_group_power_spectra, param_sampler

# FOOOF comes with some basic analysis function to work with FOOOF outputs
from fooof.analysis import get_band_peak, get_band_peak_group

In [2]:
# Reload some data and fit a FOOOF model to use
freqs = np.load('dat/freqs_lfp.npy')
spectrum = np.load('dat/spectrum_lfp.npy')
fm = FOOOF(peak_width_limits=[2, 8])
fm.fit(freqs, spectrum, [3, 30])

In [3]:
# Generate some synthetic power spectra and fit a FOOOFGroup to use
freqs, spectra, _ = gen_group_power_spectra(n_spectra=10,
                                            freq_range=[3, 40],
                                            background_params=param_sampler([[20, 2], [35, 1.5]]),
                                            gauss_params=param_sampler([[], [10, 0.5, 2]]))
fg = FOOOFGroup(peak_width_limits=[1, 8], min_peak_amplitude=0.05, max_n_peaks=6, verbose=False)
fg.fit(freqs, spectra)

### FOOOF Analysis utilities

FOOOF is packaged with minimal analysis utility functions. 

The plan is for the FOOOF module to stay this way, as supporting further analysis of FOOOF-derived results is largely outside the scope of the current module.

Here we only directly include and demonstrate utility functions covering very general use cases.

In particular, we include some utilities that are useful for parsing peak results, and extracting peaks from frequency ranges of interest. 

In [4]:
# Check out the documentation for the peak analyis helper function
get_band_peak?

### Analyzing Band-Specific Oscillations

In [5]:
# Set up indexes for accessing data, for convenience
cf_ind, am_ind, bw_ind = 0, 1, 2

In [6]:
# Define frequency bands of interest
theta_band = [4, 8]
alpha_band = [8, 12]
beta_band = [15, 30]

In [7]:
# Extract any beta band oscillations from the FOOOF model
get_band_peak(fm.peak_params_, beta_band)

array([16.08799633,  0.31497775,  2.91456728])

The 'get_band_peak' function will extract peaks within a specified band from the results of a FOOOF fit. 

You can optionally specify whether to return all oscillations within that band, or a singular result, which returns the highest power peak (if there are multiple). 

In [8]:
# Get all alpha oscillations from a FOOOFGroup object
alphas = get_band_peak_group(fg.get_all_data('peak_params'), alpha_band, len(fg))

In [9]:
# Check out some of the alpha data
alphas[0:5, :]

array([[        nan,         nan,         nan],
       [10.02967719,  0.48536385,  3.80421559],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [10.04113301,  0.49012133,  3.78016585]])

#### get_band_peak_group

Note that the design of 'get_band_peak_group' is such that it will retain information regarding which oscillation came from with model fit. 

To do so, it's output is organized such that each row corresponds to a specific model fit, such that the matrix returned is size [n_fits, 3].

For this to work, at most 1 peak is extracted for each model fit within the specified band. 

If more than 1 peak are found within the band, the peak with the highest power is extracted. If no peaks are found, that row is filled with 'nan'. 

In [10]:
# Check descriptive statistics of oscillation data
print('Alpha CF : ', np.nanmean(alphas[:, cf_ind]))
print('Alpha Amp: ', np.nanmean(alphas[:, am_ind]))
print('Alpha BW : ', np.nanmean(alphas[:, bw_ind]))

Alpha CF :  10.032316994652476
Alpha Amp:  0.48704968437097024
Alpha BW :  3.793087577369318


#### A Note on Frequency Ranges

A benefit of using FOOOF to model power spectra is that you do not have to define a priori frequency ranges from which to extract oscillations. 

Nevertheless, it may still be useful to group extracted peaks into 'bands' of interest, which is why the aforementioned functions are offered. 

Since this frequency-range selection can be done after model fitting, we do recommend checking the model results, for example by checking a histogram of the center frequencies extracted across a group, in order to ensure the frequency ranges you choose reflect the characteristics of the data under studty. 

### Analyzing the Aperiodic ('background') Signal

In [11]:
# Extract slope data from group results
sls = fg.get_all_data('background_params', 'slope')

In [12]:
# Check out slope data
sls

array([1.49974605, 2.02193707, 1.50243998, 1.50058702, 2.01839346,
       2.02086515, 1.49905775, 1.50216495, 1.52005248, 1.5156712 ])

### Comparing Across PSDs

Both of the examples above preserve information about which PSD particular features come from. If the PSDs come from across electrodes, channels or source reconstructed vertices, for example, extracting data in this way can be used to examine topographical relationships within and between these features. 

If your data comes from M/EEG, [MNE](https://github.com/mne-tools/mne-python) has visualization tools that you can use to, with a channel file and a vector of FOOOF output data, visualize FOOOF results across the scalp and/or cortex.

### Example FOOOF-related analyses

- Characterizing oscillations & aperiodic properties, and analyzing spatial topographies, across demographics, modalities, and tasks
- Comparing oscillations within and between subjects across different tasks of interest
- Predicting disease state based on FOOOF derived oscillation & aperiodic features
- Using FOOOF on a trial by trial manner to decode task properties, and behavioural states

<hr>
<div class="alert alert-success">
This is the end of the FOOOF tutorial materials!
</div>

<div class="alert alert-info">
If you are having any troubles, please submit an issue on Github [here](https://github.com/voytekresearch/fooof), and/or get in contact with us at voytekresearch@gmail.com.
</div>